## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-18-22-54-00 +0000,bbc,At least 21 killed in Spain after crash involv...,https://www.bbc.com/news/articles/cedw6ylpynyo...
1,2026-01-18-22-52-36 +0000,bbc,Government pulls Hillsborough Law debate after...,https://www.bbc.com/news/articles/c5yvldj47v9o...
2,2026-01-18-22-40-46 +0000,nypost,Extreme cold snap in NYC could bring real-feel...,https://nypost.com/2026/01/18/us-news/extreme-...
3,2026-01-18-22-38-51 +0000,nyt,"Joe Montgomery, Who Made Bicycles Lighter, Die...",https://www.nytimes.com/2026/01/18/obituaries/...
4,2026-01-18-22-38-00 +0000,nypost,JPMorgan confirms Trump claim that CEO Dimon w...,https://nypost.com/2026/01/18/business/jpmorga...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2410/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
36,trump,39
115,greenland,20
52,ice,13
348,tariffs,8
161,minnesota,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
138,2026-01-18-08-00-32 +0000,wapo,Melting ice may raise Greenland’s value. Trump...,https://www.washingtonpost.com/climate-environ...,88
184,2026-01-17-23-25-00 +0000,wsj,Trump to Hit European Nations With 10% Tariffs...,https://www.wsj.com/politics/policy/trump-to-h...,87
147,2026-01-18-06-01-39 +0000,nyt,Trump Announces New Tariffs on Some European C...,https://www.nytimes.com/2026/01/17/us/politics...,86
68,2026-01-18-17-37-12 +0000,nypost,GOP lawmakers irked at Trump’s new planned ta...,https://nypost.com/2026/01/18/us-news/gopers-i...,85
83,2026-01-18-15-00-29 +0000,cbc,European nations warn of 'dangerous downward s...,https://www.cbc.ca/news/world/eight-countries-...,83


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
138,88,2026-01-18-08-00-32 +0000,wapo,Melting ice may raise Greenland’s value. Trump...,https://www.washingtonpost.com/climate-environ...
66,33,2026-01-18-17-50-47 +0000,nypost,Minneapolis Mayor Jacob Frey rips ICE’s immigr...,https://nypost.com/2026/01/18/us-news/minneapo...
184,28,2026-01-17-23-25-00 +0000,wsj,Trump to Hit European Nations With 10% Tariffs...,https://www.wsj.com/politics/policy/trump-to-h...
38,27,2026-01-18-20-13-23 +0000,nypost,Hundreds of thousands of french fries wash up ...,https://nypost.com/2026/01/18/world-news/hundr...
133,27,2026-01-18-10-00-00 +0000,wsj,Americans are split on President Trump’s opera...,https://www.wsj.com/politics/policy/voters-nar...
98,27,2026-01-18-13-00-00 +0000,startribune,Wondering what to do indoors with kids on a co...,https://www.startribune.com/best-indoor-playgr...
185,26,2026-01-17-23-08-39 +0000,bbc,'Young 40s': Gen Z has found a new way to mock...,https://www.bbc.com/news/articles/c78vw30p1d2o...
120,26,2026-01-18-10-17-07 +0000,nypost,"Pentagon readies 1,500 troops to possibly depl...",https://nypost.com/2026/01/18/us-news/pentagon...
8,25,2026-01-18-22-25-31 +0000,bbc,Syria agrees ceasefire with Kurdish-led forces...,https://www.bbc.com/news/articles/c4gwk37ewvwo...
2,23,2026-01-18-22-40-46 +0000,nypost,Extreme cold snap in NYC could bring real-feel...,https://nypost.com/2026/01/18/us-news/extreme-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
